In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

# Save my own image

In [8]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    cv2.imshow('frame',frame)
    
    key = cv2.waitKey(33)
    if key == ord('q'):
        cv2.imwrite("myphoto.jpg",frame)
        break

cap.release()
cv2.destroyAllWindows()

# Loading Image

https://www.google.com/imgres?imgurl=http%3A%2F%2Fwww.rsa.unc.edu%2Fpsw%2Fimages%2Fwalking_figures.png&imgrefurl=http%3A%2F%2Fwww.rsa.unc.edu%2Fpsw%2F&docid=1Zqi7cTp3O9ucM&tbnid=2bzUsw4sk3CRUM%3A&vet=10ahUKEwi8mrmr_ZrXAhWGIsAKHSarAEc4ZBAzCBAoDDAM..i&w=417&h=336&safe=images&bih=671&biw=1329&as_q=pedestrian&ved=0ahUKEwi8mrmr_ZrXAhWGIsAKHSarAEc4ZBAzCBAoDDAM&iact=mrc&uact=8

In [2]:
image = cv2.imread('original_Resized.jpeg')
image = np.float32(image)/255.0

cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Preprocessing - Cropping and resizing the image

# Sobel Filter

In [3]:
sobel_imgx = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize = 1) # H
sobel_imgy = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize = 1) # V

cv2.imshow('sobel_H', sobel_imgx)
cv2.imshow('sobel_V', sobel_imgy)

cv2.waitKey(0)
cv2.destroyAllWindows()

# HOG - Calculating the mag and angle

In [4]:
mag, angle = cv2.cartToPolar(sobel_imgx, sobel_imgy, angleInDegrees=True)

In [5]:
cv2.imshow('mag',mag)
cv2.imshow('angle',angle)

cv2.waitKey(0)
cv2.destroyAllWindows()

# HOG - Calculate Histogram of Gradients in 8×8 cells

# HOG - 16×16 Block Normalization

# Calculate the HOG feature vector and send it to classifier

# Other implementation

# 1)

In [16]:
# hog = cv2.HOGDescriptor()
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
hist = hog.compute(image,winStride,padding,locations)

# 2)

https://stackoverflow.com/questions/6090399/get-hog-image-features-from-opencv-python

In [2]:
from numpy.linalg import norm

In [22]:
crop_image = image[:320, :488]

In [2]:
def myhog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bin_n = 16 # Number of bins
    bin = np.int32(bin_n*ang/(2*np.pi))

    bin_cells = []
    mag_cells = []

    cellx = celly = 8

    for i in range(0,int(img.shape[0]/celly)):
        for j in range(0,int(img.shape[1]/cellx)):
            bin_cells.append(bin[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])
            mag_cells.append(mag[i*celly : i*celly+celly, j*cellx : j*cellx+cellx])   

    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)

    # transform to Hellinger kernel
    eps = 1e-7
    hist /= hist.sum() + eps
    hist = np.sqrt(hist)
    hist /= norm(hist) + eps

    return hist, bin_cells, mag_cells

In [23]:
# imp2_hog = hog(image)
imp2_feature_vector, other_data1, other_data2 = hog(crop_image)

In [25]:
other_data1 = np.asarray(other_data1)
other_data2 = np.asarray(other_data2)
imp2_feature_vector = np.asarray(imp2_feature_vector)

print(imp2_feature_vector.shape)
print(other_data1.shape)
# cv2.imshow('imp2_hog', other_data1)
# cv2.imshow('imp22_hog', other_data2)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

(39064,)
(2440, 8, 8, 3)


In [6]:
image2 = cv2.imread('GroupAdults.png')
image2 = np.float32(image2)/255.0

sobel_imgx = cv2.Sobel(image2, cv2.CV_32F, 0, 1, ksize = 1) # H
sobel_imgy = cv2.Sobel(image2, cv2.CV_32F, 1, 0, ksize = 1) # V

mag, angle = cv2.cartToPolar(sobel_imgx, sobel_imgy, angleInDegrees=True)

outImage = cv2.add(angle, mag)
cv2.imshow('out', outImage)

# cv2.imshow('mag', mag)
# cv2.imshow('angle', angle)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
print(image2.shape)
crop_image = image2[:320, :416]

cv2.imshow('angle', crop_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

(336, 417, 3)


# Imp 3: Using scikit learn

In [3]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, color, exposure

In [5]:
hog?

In [ ]:
# image2 = cv2.imread('walking_figures.png')
image2 = cv2.imread('myphoto.jpg')


image = color.rgb2gray(image2)
# image = color.rgb2gray(data.astronaut())

fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualise=True)

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

# ax1.axis('off')
# ax1.imshow(image, cmap=plt.cm.gray)
# ax1.set_title('Input image')
# ax1.set_adjustable('box-forced')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 0.02))

# ax2.axis('off')
# ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
# ax2.set_title('Histogram of Oriented Gradients')
# ax1.set_adjustable('box-forced')
# plt.show()

cv2.imshow('angle', hog_image_rescaled)

cv2.waitKey(0)
cv2.destroyAllWindows()

/home/ali/.local/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
